# Vertex Tabular Binary Classification with .CustomTrainingJob()

<center><img src="../images/03.png"/></center>

## Set Constants

In [60]:
PROJECT_ID = 'jchavezar-demo'
REGION = 'us-central1'
DATASET_URI = 'gs://vtx-datasets-public/ecommerce/datasets.csv'
MODEL_URI = 'gs://vtx-models/pytorch/ecommerce'
STAGING_URI = 'gs://vtx-staging/pytorch/ecommerce/'
TRAIN_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-11:latest'
PREDICTION_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest'

## Create Folder Structure

```
source
     └─── trainer
          |  train.py
          |

```

In [45]:
!rm -fr source
!mkdir -p source/trainer

## Intro

Below we have the code for the training, it was made with PyTorch by building a neural network with these components:

- 2 types of features set: categorical and numerical.
- Shape detection of embedding layer for categorical.
- Drouput to avoid overfit during the training.
- Batch Normalization to standarize the data.
- 1 input layer, shape: 114x32: 
  - 114 is the number of total features (categorical and numerical) after the embedding.
  - 32 is the number of the neurons.
- Activation function applied to the last input layer to fix non-linearity.
- 1 output layer, shape: 32x2.

The following diagram shows the neural netowkr with steps ordered used during the Model building class: ShelterOutcomeModel.

<center><img src="../images/04-pytorch-nn.png"/></center>

In [62]:
%%writefile source/trainer/train.py

import os
import torch
import argparse
import numpy as np
import pandas as pd
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
from google.cloud import storage
import torch.optim as torch_optim
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

parser = argparse.ArgumentParser()

parser.add_argument(
    '--dataset_uri',
    type = str,
    help = 'Dataset uri in the format gs://[BUCKET]/*suffix*/file_name.extension')
parser.add_argument(
    '--project',
    type = str,
    help = 'This is the tenant or the Google Cloud project id name')

args = parser.parse_args()

## Prepare Data
df = pd.read_csv(args.dataset_uri)

target = 'will_buy_on_return_visit'
cat_columns = [i for i in df.columns if df[i].dtypes == 'object']
num_columns = [i for i in df.columns if df[i].dtypes == 'int64' or df[i].dtypes == 'float']
num_columns.remove(target)

cat_train_df = df[cat_columns]
num_train_df = df[num_columns]
label = df[target].to_numpy()

## Encoding

for col in cat_train_df.columns:
    cat_train_df[col] = LabelEncoder().fit_transform(cat_train_df[col])
    cat_train_df[col] = cat_train_df[col].astype('category')

train_df = pd.concat([cat_train_df,num_train_df], axis=1)
X_train, X_val, y_train, y_val = train_test_split(train_df, label, test_size=0.10, random_state=0)

# Categorical Embedding

embedded_cols = {n: len(col.cat.categories) for n,col in X_train[cat_columns].items() if len(col.cat.categories) > 2}
embedded_col_names = embedded_cols.keys()
embedding_sizes = [(n_categories, min(50, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]


## Numerical columns standarization

scaler = StandardScaler()
X_train[num_columns] = scaler.fit_transform(X_train[num_columns])
X_val[num_columns] = scaler.transform(X_val[num_columns])

## PyTorch Dataset

class ShelterOutcomeDataset(Dataset):
    def __init__(self, X, Y, embedded_col_names):
        X = X.copy()
        self.X1 = X.loc[:,embedded_col_names].copy().values.astype(np.int64) #categorical columns
        self.X2 = X.drop(columns=embedded_col_names).copy().values.astype(np.float32) #numerical columns
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]
    
## Train and Valid datasets

train_ds = ShelterOutcomeDataset(X_train, y_train, embedded_col_names)
valid_ds = ShelterOutcomeDataset(X_val, y_val, embedded_col_names)

## CPU or GPU selection

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()


## Model

class ShelterOutcomeModel(nn.Module):
    def __init__(self, embedding_sizes, n_cont):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings) #length of all embeddings combined
        self.n_emb, self.n_cont = n_emb, n_cont
        self.lin1 = nn.Linear(self.n_emb + self.n_cont, 32)
        self.lin2 = nn.Linear(32, 2)
        self.bn1 = nn.BatchNorm1d(self.n_cont)
        self.bn2 = nn.BatchNorm1d(32)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
        

    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = self.lin2(x)
        return x
    
model = ShelterOutcomeModel(embedding_sizes, 4)
to_device(model, device)

## Define Optimizer

def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

## Train Model

def train_model(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x1, x2, y in train_dl:
        batch = y.shape[0]
        output = model(x1, x2)
        loss = F.cross_entropy(output, y)   
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total


def val_loss(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    for x1, x2, y in valid_dl:
        current_batch_size = y.shape[0]
        out = model(x1, x2)
        loss = F.cross_entropy(out, y)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        pred = torch.max(out, 1)[1]
        correct += (pred == y).float().sum().item()
    print("valid loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
    return sum_loss/total, correct/total

def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs): 
        loss = train_model(model, optim, train_dl)
        print("training loss: ", loss)
        val_loss(model, valid_dl)
        
        
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size,shuffle=True)

train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)


train_loop(model, epochs=8, lr=0.05, wd=0.00001)
torch.save(model, 'model.pt')

bucket = os.environ['AIP_MODEL_DIR'].split('/')[2]
blob_name = '/'.join(os.environ['AIP_MODEL_DIR'].split('/')[3:])

storage_client = storage.Client(project=args.project)
bucket = storage_client.bucket(bucket)
blob = bucket.blob(blob_name+'model.pt')
blob.upload_from_filename('model.pt')

Overwriting source/trainer/train.py


## Training Job (CustomJob.from_local_script)

To speed up the training a GPU NVIDIA Tesla T4 is used, it should take around 2 minutes to finish.

In [63]:
from google.cloud import aiplatform as aip

customJob = aip.CustomJob.from_local_script(
    display_name = 'pytorch_tab_sa_ecommerce',
    script_path = 'source/trainer/train.py',
    container_uri = TRAIN_IMAGE_URI,
    requirements = ['scikit-learn'],
    args = [
        '--dataset_uri', 
        DATASET_URI,
        '--project',
        PROJECT_ID
    ],
    replica_count = 1,
    machine_type = 'n1-standard-4',
    accelerator_type = 'NVIDIA_TESLA_T4',
    accelerator_count = 1,
    staging_bucket = STAGING_URI,
    base_output_dir = MODEL_URI
)

customJob.run()

Training script copied to:
gs://vtx-staging/pytorch/ecommerce/aiplatform-2022-11-21-17:09:53.367-aiplatform_custom_trainer_script-0.1.tar.gz.
Creating CustomJob
CustomJob created. Resource name: projects/569083142710/locations/us-central1/customJobs/5323474386999574528
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/569083142710/locations/us-central1/customJobs/5323474386999574528')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5323474386999574528?project=569083142710
CustomJob projects/569083142710/locations/us-central1/customJobs/5323474386999574528 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/5323474386999574528 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/customJobs/5323474386999574528 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/569083142710/locations/us-central1/cus